In [ ]:
import pandas as pd
import os
import re
from tqdm import tqdm

In [ ]:
example = pd.read_json("model_judgment/claude-3-5-sonnet-20240620/claude-2.1.jsonl", lines=True)
example.head()

In [ ]:
example["games"][0]

In [ ]:
example["games"][0][0]["user_prompt"]

In [ ]:
example["games"][0][0]["user_prompt"][15:1180].strip()

In [ ]:
def file_preprocess(file_path):
    final_data = pd.DataFrame(columns=["id", "prompt", "response_a", "response_b", "winner_model_a", "winner_model_b", "winner_model_tie", "answer_model", "judge_model"])
    data = pd.read_json(file_path, lines=True)
    for idx, row in tqdm(data.iterrows(), total=len(data)):
        for judge in row["games"]:
            try:
                text = judge["user_prompt"]
                prompt = text[15: text.find('<|The Start of Assistant A\'s Answer|>')].strip()
                response_a = text[text.find('<|The Start of Assistant A\'s Answer|>') + len('<|The Start of Assistant A\'s Answer|>'): text.find('<|The End of Assistant A\'s Answer|>')].strip()
                response_b = text[text.find('<|The Start of Assistant B\'s Answer|>') + len('<|The Start of Assistant B\'s Answer|>'): text.find('<|The End of Assistant B\'s Answer|>')].strip()
                if judge["score"] == "A=B":
                    winner_model_a = 0
                    winner_model_b = 0
                    winner_model_tie = 1
                elif judge["score"] == "A>B" or judge["score"] == "B<A":
                    winner_model_a = 1
                    winner_model_b = 0
                    winner_model_tie = 0
                elif judge["score"] == "A<B" or judge["score"] == "B>A":
                    winner_model_a = 0
                    winner_model_b = 1
                    winner_model_tie = 0
                new_row = {
                    "id": row["question_id"],
                    "prompt": [prompt],
                    "response_a": [response_a],
                    "response_b": [response_b],
                    "winner_model_a": winner_model_a,
                    "winner_model_b": winner_model_b,
                    "winner_model_tie": winner_model_tie,
                    "answer_model": row["model"],
                    "judge_model": row["judge"]
                }
                new_row_df = pd.DataFrame([new_row], columns=final_data.columns)
                final_data = pd.concat([final_data, new_row_df], ignore_index=True)
            except:
                print("Error")
                continue
    print(type(final_data["prompt"][0]))
    return final_data

In [ ]:
file_preprocess("model_judgment/claude-3-5-sonnet-20240620/claude-2.1.jsonl")

In [ ]:
folder_paths = ["model_judgment/claude-3-5-sonnet-20240620", "model_judgment/claude-3-opus-20240229", "model_judgment/gemini-1.5-pro-api-0514", "model_judgment/gpt-4-1106-preview", "model_judgment/llama-3-70b-instruct"]

possible_leak_data = pd.DataFrame(columns=["id", "prompt", "response_a", "response_b", "winner_model_a", "winner_model_b", "winner_model_tie", "answer_model", "judge_model"])

for folder_path in folder_paths:
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path) and file_name.endswith(".jsonl"):
            final_data = file_preprocess(file_path)
            possible_leak_data = pd.concat([possible_leak_data, final_data], ignore_index=True)

print(possible_leak_data.shape)